In [1]:
import numpy as np
import pandas as pd
import re
import os
from table import Table

In [2]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [3]:
tables = {}

directory = "tables/"
for filename in os.listdir(directory):
    if filename.endswith(".xls"):
        file_directory = os.path.join(directory, filename)
        table = Table(file_directory, "2019")
        tables[table.id] = table

In [4]:
tables

{'203.10': <table.Table at 0x24895eb2220>,
 '203.50': <table.Table at 0x2489802ba00>,
 '204.10': <table.Table at 0x24894dd7280>,
 '204.27': <table.Table at 0x24894e72610>,
 '213.10': <table.Table at 0x24898394190>,
 '215.10': <table.Table at 0x248983c05e0>,
 '217.10': <table.Table at 0x248986d8490>,
 '217.15': <table.Table at 0x248986c1910>,
 '219.70': <table.Table at 0x24898770610>,
 '236.30': <table.Table at 0x248987a5e20>,
 '318.45': <table.Table at 0x2489890c310>}

In [22]:
tab = tables['203.50']

In [23]:
tab.row_info

,digest_table_id,digest_table_year,digest_table_sub_id,digest_table_sub_title,row_index,row_level_1,row_ref_note_1,row_level_2,row_ref_note_2,row_level_3,row_ref_note_3,row_level_4,row_ref_note_4,row_level_5,row_ref_note_5,row_level_6,row_ref_note_6,row_level_7,row_ref_note_7,is_total,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P
5,203.50,2019,A,,1,United States,,1995.0,,,,,,,,,,,,FALSE,44840.5,29044.1,7550.57,6072.15,1668.4389999999999\1\,---,505.2,---,100,64.7721,16.8387,13.5417,3.72083207582006\1\,---,1.12666,---
6,203.50,2019,A,,2,United States,,2000.0,,,,,,,,,,,,FALSE,47203.5,28877.5,8099.79,7725.84,1950.33\1\,---,550.03,---,100,61.1767,17.1593,16.3671,4.131745291385885\1\,---,1.16523,---
7,203.50,2019,A,,3,United States,,2001.0,,,,,,,,,,,,FALSE,47671.8,28734.5,8176.92,8169.18,2027.612\1\,---,563.586,---,100,60.2757,17.1525,17.1363,4.253269587248808\1\,---,1.18222,---
8,203.50,2019,A,,4,United States,,2002.0,,,,,,,,,,,,FALSE,48183.1,28618.4,8298.86,8594.17,2088.478\1\,---,583.18,---,100,59.3951,17.2236,17.8365,4.334462927509458\1\,---,1.21034,---
9,203.50,2019,A,,5,United States,,2003.0,,,,,,,,,,,,FALSE,48540.2,28442.5,8348.81,9010.91,2145.2200000000003\1\,---,592.789,---,100,58.5957,17.1998,18.5638,4.419469505852003\1\,---,1.22123,---
10,203.50,2019,A,,6,United States,,2004.0,,,,,,,,,,,,FALSE,48795.5,28318.1,8386.22,9316.7,2183.1810000000005\1\,---,591.232,---,100,58.0344,17.1865,19.0934,4.474147341356416\1\,---,1.21165,---
11,203.50,2019,A,,7,United States,,2005.0,,,,,,,,,,,,FALSE,49113.3,28005.2,8444.77,9786.8,2278.5150000000003\1\,---,598.029,---,100,57.0216,17.1945,19.927,4.6393035955353685\1\,---,1.21765,---
12,203.50,2019,A,,8,United States,,2006.0,,,,,,,,,,,,FALSE,49315.8,27801.2,8422.15,10165.6,2331.6809999999996\1\,---,595.262,---,100,56.3737,17.078,20.6132,4.728056757096431\1\,---,1.20704,---
13,203.50,2019,A,,9,United States,,2007.0,,,,,,,,,,,,FALSE,49290.6,27454.4,8391.78,10453.8,2396.455\1\,---,594.148,---,100,55.6991,17.0251,21.2085,4.861894546580411\1\,---,1.2054,---
14,203.50,2019,A,,10,United States,,2008.0,,,,,,,,,,,,FALSE,49265.6,27057.1,8358.07,10563.1,2405.106,46.358,589.327,246.534\2\,100,54.9209,16.9653,21.4411,4.881920380423067,0.0940982,1.19622,0.5004184260765308\2\


In [25]:
tab.col_info

,digest_table_id,digest_table_year,column_index,column_level_1,column_ref_note_1,column_level_2,column_ref_note_2,column_level_3,column_ref_note_3,column_level_4,column_ref_note_4,column_level_5,column_ref_note_5,column_level_6,column_ref_note_6,column_level_7,column_ref_note_7
0,203.50,2019,A,Enrollment (in thousands),,Total,,,,,,,,,,,
1,203.50,2019,B,Enrollment (in thousands),,White,,,,,,,,,,,
2,203.50,2019,C,Enrollment (in thousands),,Black,,,,,,,,,,,
3,203.50,2019,D,Enrollment (in thousands),,Hispanic,,,,,,,,,,,
4,203.50,2019,E,Enrollment (in thousands),,Asian,,,,,,,,,,,
5,203.50,2019,F,Enrollment (in thousands),,Pacific Islander,,,,,,,,,,,
6,203.50,2019,G,Enrollment (in thousands),,American Indian/ Alaska Native,,,,,,,,,,,
7,203.50,2019,H,Enrollment (in thousands),,Two or more races,,,,,,,,,,,
8,203.50,2019,I,Percentage distribution,,Total,,,,,,,,,,,
9,203.50,2019,J,Percentage distribution,,White,,,,,,,,,,,


## Extract footnotes, create cell_info table

- loop through each value in data
- if it has a footnote:
 - extract id, year, sub_id, sub_title, row_index from row_info
 - extract column_index
 - lookup cell_ref_note in footnotes
- if it is a special char:
 - lookup special note

In [92]:
data = tab.row_info.loc[:, 'A':]

In [93]:
data

,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P
5,44840.5,29044.1,7550.57,6072.15,1668.4389999999999\1\,---,505.2,---,100,64.7721,16.8387,13.5417,3.72083207582006\1\,---,1.12666,---
6,47203.5,28877.5,8099.79,7725.84,1950.33\1\,---,550.03,---,100,61.1767,17.1593,16.3671,4.131745291385885\1\,---,1.16523,---
7,47671.8,28734.5,8176.92,8169.18,2027.612\1\,---,563.586,---,100,60.2757,17.1525,17.1363,4.253269587248808\1\,---,1.18222,---
8,48183.1,28618.4,8298.86,8594.17,2088.478\1\,---,583.18,---,100,59.3951,17.2236,17.8365,4.334462927509458\1\,---,1.21034,---
9,48540.2,28442.5,8348.81,9010.91,2145.2200000000003\1\,---,592.789,---,100,58.5957,17.1998,18.5638,4.419469505852003\1\,---,1.22123,---
10,48795.5,28318.1,8386.22,9316.7,2183.1810000000005\1\,---,591.232,---,100,58.0344,17.1865,19.0934,4.474147341356416\1\,---,1.21165,---
11,49113.3,28005.2,8444.77,9786.8,2278.5150000000003\1\,---,598.029,---,100,57.0216,17.1945,19.927,4.6393035955353685\1\,---,1.21765,---
12,49315.8,27801.2,8422.15,10165.6,2331.6809999999996\1\,---,595.262,---,100,56.3737,17.078,20.6132,4.728056757096431\1\,---,1.20704,---
13,49290.6,27454.4,8391.78,10453.8,2396.455\1\,---,594.148,---,100,55.6991,17.0251,21.2085,4.861894546580411\1\,---,1.2054,---
14,49265.6,27057.1,8358.07,10563.1,2405.106,46.358,589.327,246.534\2\,100,54.9209,16.9653,21.4411,4.881920380423067,0.0940982,1.19622,0.5004184260765308\2\


In [94]:
col_list = [
    'digest_table_id',
    'digest_table_year',
    'digest_table_sub_id',
    'digest_table_sub_title',
    'row_index',
    'column_index',
    'cell_ref_note',
    'cell_special_note',
]
cell_info = pd.DataFrame(columns=col_list)

In [95]:
symbols = ['---', '†', '#', '!', '‡', '*']
footnotes = ['Not available.',
             'Not applicable.',
             'Rounds to zero.',
             'Interpret data with caution. The coefficient of variation (CV) for this estimate is between 30 and 50 percent.',
             'Reporting standards not met. Either there are too few cases for a reliable estimate or the coefficient of variation (CV) for this estimate is 50 percent or greater.',
             'p < .05 significance level.'
            ]

symbol_dict = dict(zip(symbols, footnotes))

In [96]:
for row in data.index:
    for col in data.columns:
        
        cell_val = str(data.loc[row, col])
        
        has_fn = re.match(r".*\\([0-9])\\$", cell_val)
        is_spec = cell_val in symbols
        
        ref_note = ""
        spec_note = ""
        
        if has_fn:
            ref_note = has_fn.group(1)
        if is_spec:
            spec_note = symbol_dict[cell_val]
        if has_fn or is_spec:
            l = list(tab.row_info.loc[row, ['digest_table_id',
                               'digest_table_year',
                               'digest_table_sub_id',
                               'digest_table_sub_title',
                               'row_index']].values) + [col, ref_note, spec_note]
            df_row = pd.DataFrame(l, index=col_list).T
            cell_info = cell_info.append(df_row, ignore_index=True)

In [97]:
cell_info

,digest_table_id,digest_table_year,digest_table_sub_id,digest_table_sub_title,row_index,column_index,cell_ref_note,cell_special_note
0,203.50,2019,A,,1,E,1,
1,203.50,2019,A,,1,F,,Not available.
2,203.50,2019,A,,1,H,,Not available.
3,203.50,2019,A,,1,M,1,
4,203.50,2019,A,,1,N,,Not available.
5,203.50,2019,A,,1,P,,Not available.
6,203.50,2019,A,,2,E,1,
7,203.50,2019,A,,2,F,,Not available.
8,203.50,2019,A,,2,H,,Not available.
9,203.50,2019,A,,2,M,1,


In [98]:
cell_info['cell_ref_note'] = cell_info['cell_ref_note'].replace(tab.footnotes)
cell_info

,digest_table_id,digest_table_year,digest_table_sub_id,digest_table_sub_title,row_index,column_index,cell_ref_note,cell_special_note
0,203.50,2019,A,,1,E,Includes Pacific Islanders.,
1,203.50,2019,A,,1,F,,Not available.
2,203.50,2019,A,,1,H,,Not available.
3,203.50,2019,A,,1,M,Includes Pacific Islanders.,
4,203.50,2019,A,,1,N,,Not available.
5,203.50,2019,A,,1,P,,Not available.
6,203.50,2019,A,,2,E,Includes Pacific Islanders.,
7,203.50,2019,A,,2,F,,Not available.
8,203.50,2019,A,,2,H,,Not available.
9,203.50,2019,A,,2,M,Includes Pacific Islanders.,
